<a href="https://colab.research.google.com/github/raruggie/GRRIEn/blob/main/tutorials/detecting-changes-in-sentinel-1-imagery-pt-1/CEE609_data_download_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [9]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3u0K0TPtY7AmStdGT_5RKl4liZlggRbF-wpmDRq7mxA&tc=od3aSaZWfABRaAiX73nDEmo9fb5mEMRpZfvLS30HxaE&cc=qXuDlGA8NIot2qR5j2MTqINNdeVge9J6EZzRz-yHEng

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdeyEhDk3IL9Dc66JZ6-0Vx_TWEzoTb7gRqpWc6sezwJY4nxGsM1YK0

Successfully saved authorization token.


In [11]:
#@title Install packages for Google Colab

%%capture
!pip install rasterio
!pip install earthpy
import os
import itertools
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import plotting_extent
import numpy as np
import earthpy as et
import earthpy.plot as ep
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

In [12]:
#@title Function to convert ee array to panda df
# need this function for below
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [13]:
#@title Functions for Exogenous Organic Matter Indices (EOMI) band indices
def addEOMI1(image):
    EVI = image.expression('(B11-B8A)/(B11+B8A)', {
        'B11' : image.select('B11'),
        'B8A' : image.select('B8A')}).rename('EVI')
    return image.addBands(EVI)


def EOMI1(B11, B8A):
    """Calculated band index for EOMI"""
    return (B11-B8A)/(B11+B8A)

def EOMI2(B12, B4):
    """Calculated band index for EOM2"""
    x =  (B12-B4)/(B12+B4)
    return x

def EOMI3(B11, B8A, B12, B4):
    """Calculated band index for EOM3"""
    x =  ((B11-B8A)+(B12-B4))/((B11+B8A+B12+B4))
    return x

def EOMI4(B11, B4):
    """Calculated band index for EOM4"""
    x =  (B11-B4)/(B11+B4)
    return x

def NBR2(B11, B12):
    """Calculated band index for NBR2"""
    x =  (B11-B12)/(B11+B12)
    return x

In [14]:
#@title Functions for messing with s2cloudless proability image collection:

# intialize parameters used in functions
CLOUD_FILTER = 90
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

# Define a function to filter the SR and s2cloudless collections according to area of interest and date parameters, 
# then join them on the system:index property. The result is a copy of the SR collection where each image has a new 
# 's2cloudless' property whose value is the corresponding s2cloudless image
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))
  
# Define a function to add dark pixels, cloud projection, and identified shadows as bands to an S2 SR image input. 
# Note that the image input needs to be the result of the above add_cloud_bands function because it relies on knowing which pixels are 
# considered cloudy ('clouds' band)
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Define a function to assemble all of the cloud and cloud shadow components and produce the final mask.
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)
    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# Define a function to apply the cloud mask to each image in the collection.
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Define functions to display image and mask component layers.
# Folium will be used to display map layers. Import folium and define a method to display Earth Engine image tiles.
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Define a function to display all of the cloud and cloud shadow components to an interactive Folium map. 
# The input is an image collection where each image is the result of the add_cld_shdw_mask function defined previously
def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()
    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=13)
    # Add layers to the folium map.
    m.add_ee_layer(img,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},'S2 image', True, 1, 9)
    m.add_ee_layer(probability,{'min': 0, 'max': 100},'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds, {'palette': 'e056fd'}, 'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform, {'min': 0, 'max': 1, 'palette': ['white', 'black']}, 'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels, {'palette': 'orange'}, 'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'}, 'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},'cloudmask', True, 0.5, 9)
    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())
    # Display the map.
    display(m)

In [15]:
#@title Geojson for field aois
# AHS and DC
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.31939702284171,
              44.15031909900094
            ],
            [
              -73.31499969430045,
              44.150539958983614
            ],
            [
              -73.3151316141566,
              44.15101322759219
            ],
            [
              -73.31442804159038,
              44.151107880858405
            ],
            [
              -73.3146479080171,
              44.15199130402354
            ],
            [
              -73.31526353401328,
              44.15240146028401
            ],
            [
              -73.31605505315063,
              44.15230680909241
            ],
            [
              -73.31631889286288,
              44.153474163180505
            ],
            [
              -73.31517558744216,
              44.153663461667435
            ],
            [
              -73.31561532029615,
              44.15543021826613
            ],
            [
              -73.31675862571684,
              44.155998093082275
            ],
            [
              -73.31715438528579,
              44.155998093082275
            ],
            [
              -73.31737425171306,
              44.15498853407544
            ],
            [
              -73.3179019311376,
              44.15473614162491
            ],
            [
              -73.31768206471087,
              44.153821209942976
            ],
            [
              -73.3179019311376,
              44.15334796385173
            ],
            [
              -73.31886934341718,
              44.15284316383796
            ],
            [
              -73.31979278241064,
              44.1520228546064
            ],
            [
              -73.31939702284171,
              44.15031909900094
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.3582447745077,
              44.143810086322816
            ],
            [
              -73.35799464344306,
              44.14345635041016
            ],
            [
              -73.35772490610948,
              44.143241279968976
            ],
            [
              -73.35715546062814,
              44.143198265786594
            ],
            [
              -73.35711050440575,
              44.14287565842099
            ],
            [
              -73.35678082544304,
              44.14277887586772
            ],
            [
              -73.356900708702,
              44.142402497762504
            ],
            [
              -73.35691569410945,
              44.14207988604832
            ],
            [
              -73.35684076707214,
              44.14184330300441
            ],
            [
              -73.35685575247962,
              44.14162822668689
            ],
            [
              -73.35720041685052,
              44.1414991805205
            ],
            [
              -73.35751511040576,
              44.14128410294927
            ],
            [
              -73.35773989151696,
              44.140595849457185
            ],
            [
              -73.35785977477666,
              44.14006889995392
            ],
            [
              -73.35717044603558,
              44.139746275487965
            ],
            [
              -73.35066677921822,
              44.1403700145315
            ],
            [
              -73.35101062208136,
              44.142072309106425
            ],
            [
              -73.3512204177851,
              44.143212196086125
            ],
            [
              -73.35270397311872,
              44.14305089266176
            ],
            [
              -73.35306684447445,
              44.145149409871834
            ],
            [
              -73.35327662997008,
              44.14668946420335
            ],
            [
              -73.3535313818962,
              44.1480658210873
            ],
            [
              -73.35230257848795,
              44.14815184232668
            ],
            [
              -73.3525123741917,
              44.14942064104943
            ],
            [
              -73.3560964692142,
              44.14912893206903
            ],
            [
              -73.35819442625233,
              44.148935387307944
            ],
            [
              -73.35862900306724,
              44.14884936720992
            ],
            [
              -73.35870393010455,
              44.14861281129507
            ],
            [
              -73.35876387173367,
              44.14820421248106
            ],
            [
              -73.35892871121575,
              44.148021417359104
            ],
            [
              -73.359243404771,
              44.147935395929636
            ],
            [
              -73.35961803995684,
              44.14784937437426
            ],
            [
              -73.35984282106803,
              44.14772034180669
            ],
            [
              -73.35987818391523,
              44.14697151244681
            ],
            [
              -73.35987818391523,
              44.14645537280754
            ],
            [
              -73.35996809635998,
              44.145724167260084
            ],
            [
              -73.3600879796197,
              44.14497144620606
            ],
            [
              -73.36011795043387,
              44.14428323569982
            ],
            [
              -73.359248796804,
              44.143831593195046
            ],
            [
              -73.3582447745077,
              44.143810086322816
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
# general Panton area
geoJSON_gen = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -73.39192033862791,
              44.17172464979126
            ],
            [
              -73.39192033862791,
              44.11449266900607
            ],
            [
              -73.28488148069843,
              44.11449266900607
            ],
            [
              -73.28488148069843,
              44.17172464979126
            ],
            [
              -73.39192033862791,
              44.17172464979126
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
# save geojson as ee geometery class

# general panton area
coords_gen = geoJSON_gen['features'][0]['geometry']['coordinates']
aoi_gen = ee.Geometry.Polygon(coords_gen)

# AHS and DC fields
coords_DC = geoJSON['features'][0]['geometry']['coordinates']
coords_AHS = geoJSON['features'][1]['geometry']['coordinates']
aoi_DC = ee.Geometry.Polygon(coords_DC)
aoi_AHS = ee.Geometry.Polygon(coords_AHS)

Sentinel-2 Cloud Masking with s2cloudless
https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

In [118]:
#@title DC 2019 pre

# Define collection filter and cloud mask parameters
AOI = aoi_DC
START_DATE = '2019-09-29'
END_DATE = '2019-10-11'

# get pre collection

c1 = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# Add cloud and cloud shadow component bands to each image and then apply the mask to each image. 
# Reduce the collection by median (in your application, you might consider using medoid reduction to build
# a composite from actual data values, instead of per-band statistics).
# Actually ended up using getRegion because the image resulting from .median.clip was not working

img = c1.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median().clip(AOI)

# export IMAGE object to tif in drive

task_config = {'image': img,'fileFormat': 'GeoTIFF','folder': 'data','description': "DC 2019 pre",'scale':20,'region':AOI}
task = ee.batch.Export.image.toDrive(**task_config )
task.start()
task.status()


{'state': 'READY',
 'description': 'DC 2019 pre',
 'creation_timestamp_ms': 1671138828289,
 'update_timestamp_ms': 1671138828289,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'VB5S7Q3BZKAABQXNTTH7RRHW',
 'name': 'projects/earthengine-legacy/operations/VB5S7Q3BZKAABQXNTTH7RRHW'}

In [81]:
#@title Scratch cell for python magic
pwd

'/content/drive/MyDrive/data'

In [105]:
#@title Map the image from above code cell using Folium
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=14)

# Add layers to the folium map.
m.add_ee_layer(img,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

Clearly the cloudy cover percentage property doesn't apply to the aoi because when  Iset ascending to false in the sort function the image has less clouds of AHS. 

Moving along, will come back to cloudy pixel issue later. So assume you have the least cloudy images available to you...

In [ ]:
#@title Find out dates of best images using geemap
# # find out dates of best images
# print(geemap.image_props(s2_col_pre).get('IMAGE_DATE').getInfo())
# print(geemap.image_props(s2_col_post).get('IMAGE_DATE').getInfo())
# # print(geemap.image_props(s2_col_pre).getInfo())

In [ ]:
#@title Left over code from S2 cloud proability workflow

# # apply functionv to merge S2 SR and S2 cloud probability image collections
# # this adds a 's2cloudless' property whose value is the corresponding s2cloudless image to the images in the S2 SR image collection
# pre = get_s2_sr_cld_col(AOI, START_DATE, END_DATE).first()
# landsat_props = geemap.image_props(pre)
# print(landsat_props.getInfo())

# # # Map the add_cld_shdw_mask function over the collection to add mask component bands to each image.
# # s2_sr_cld_col_eval_disp = pre.map(add_cld_shdw_mask)
# s2_sr_cld_col_eval_disp = add_cld_shdw_mask(pre)
# landsat_props = geemap.image_props(s2_sr_cld_col_eval_disp)
# print(landsat_props.getInfo())

# # sort by 


# 
# print('Collection without properties', s2_sr_cld_col_eval_disp)
# lst = pre.getRegion(aoi_AHS, 20).getInfo()
# lst[:1]
# names = [item[0] for item in lst]
# set(names)



# # Display mask component layers
# # Map the add_cld_shdw_mask function over the collection to add mask component bands to each image, then display the results.
# # Give the system some time to render everything, it should take less than a minute.
# s2_sr_cld_col_eval_disp = pre.map(add_cld_shdw_mask)
# # display_cloud_layers(s2_sr_cld_col_eval_disp)

# # filter images in collections for the ones with the lease clouds
# # to do this I will look at the sum of the three new bands for each image, clouds, dark_pixels, and shadows
# # the images with the lowest sums will have the most data over the fields, and will be the best images to use
# lst = s2_sr_cld_col_eval_disp.getRegion(aoi_AHS, 20).getInfo()
# # print(lst[:5])
# # names = [item[0] for item in lst]
# # set(names)

# # split up list of list by unqiue values in ID column
# # d = [list(x) for _,x in itertools.groupby(lst,lambda x:x[0])]
# # d[:5]

# # convert to pandas datafame using function from above
# pdf = ee_array_to_df(lst,['clouds', 'dark_pixels', 'shadows'])

# # calcualte the number of no-good pixels in each image
# pdf['pixels'] = 1 # create cell to check if all dates have the same number of pixels
# pdf['sum'] = pdf.iloc[:, 2:5].sum(axis=1) # remeber panads last index in not inclusive
# pdf = pdf.rename(columns={'datetime': 'date'})
# # pdf.dtypes # check column classes
# pdf['date'] = pd.to_datetime(pdf['date']).dt.date
# pdf = pdf.drop('time', axis=1)
# pdf = pdf.groupby('date')['clouds', 'dark_pixels', 'shadows', 'sum', 'pixels'].sum()
# print(pdf.head(20))

# # check that pixel count equal field size - AHS is 14.16 ha
# 1818*(20)*0.0001 # number if pixels * spatial resolution in m2 * conversion from m2 to ha

In [117]:
# # image collection, filter over AHS aoi
# col = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(aoi_gen)
# # select bands
# col = col.select(['B12', 'B11', 'B8A', 'B4'])
# # filter date - start with general date to narrow collection
# col = col.filterDate('2018-10-01','2020-11-01')
# a19_pre_list
# get the data for the pixels over AHS field
# col_list = col.getRegion(aoi_AHS, 20).getInfo() # not sure what scale to use here, assume it is of largest resolution of the listed bands
# also, need to filter farther first as this is too big for nlist object in py...
# a18_pre_list=col.filterDate('2018-10-01','2018-10-15').getRegion(aoi_AHS, 20).getInfo()
# a18_post_list=col.filterDate('2018-10-17', '2018-10-30').getRegion(aoi_AHS, 20).getInfo()
# this lists no bands in collection, proably no images in 2018
# trying 2019
# a19_pre_list=col.filterDate('2019-10-01','2019-10-15').getRegion(aoi_AHS, 20).getInfo()
# a19_post_list=col.filterDate('2019-10-17', '2019-10-30').getRegion(aoi_AHS, 20).getInfo()
# # preview resulting list
# len(a19_pre_list)
# a19_pre_list[:5]
# # convert to pandas datafame using function from above
# a19_pre_df = ee_array_to_df(a19_pre_list,['B12', 'B11', 'B8A', 'B4'])
# a19_post_df = ee_array_to_df(a19_post_list,['B12', 'B11', 'B8A', 'B4'])
# # # look at df
# print(a19_pre_df.head(20))
# print(a19_post_df.head(20))
# there are 5 images for pre and 5 for post
# need to do some sort of cloud analysis
# S2A product has a 60meter colud mask band, but the bands I selected above are 10 and 20m. I think 60 


# # calculate EOI indexs from Dodin et al. 2021
# x=a19_pre_df
# y=a19_post_df
# x['EOMI1']=(x['B11']-x['B8A'])/(x['B11']+x['B8A']) # why doesn't this work: a19_pre_df.apply(lambda x: EOMI1(a19_pre_df['B11'], a19_pre_df['B8A']), axis=1) 
# x['EOMI2']=(x['B12']-x['B4'])/(x['B12']+x['B4'])
# x['EOMI3']=((x['B11']-x['B8A'])+(x['B12']-x['B4']))/(x['B11']+x['B8A']+x['B12']+x['B4'])
# x['EOMI4']=(x['B11']-x['B4'])/(x['B11']+x['B4'])
# x['NBR2']=(x['B11']-x['B12'])/(x['B11']+x['B12'])
# y['EOMI1']=(y['B11']-y['B8A'])/(y['B11']+y['B8A']) 
# y['EOMI2']=(y['B12']-y['B4'])/(y['B12']+y['B4'])
# y['EOMI3']=((y['B11']-y['B8A'])+(y['B12']-y['B4']))/(y['B11']+y['B8A']+y['B12']+y['B4'])
# y['EOMI4']=(y['B11']-y['B4'])/(y['B11']+y['B4'])
# y['NBR2']=(y['B11']-y['B12'])/(y['B11']+y['B12'])
# plot histograms
# x.iloc[:,-5:].hist()
# y.iloc[:,-5:].hist()
# will filter the stack into multiple variables for each year of manure application
# a18_pre=col.filterDate('2018-10-01','2018-10-15') 
# a18_post=col.filterDate('2018-10-17', '2018-10-30')

##  extra code
# band_arrs = c1.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).median().clip(AOI).sample() # dont use this one for now
# img_for_band_names = c1.map(add_cld_shdw_mask).map(apply_cld_shdw_mask).getRegion(aoi_AHS, 20).getInfo() # note I have no idea how t control which image I get. 
# band_names = list(np.concatenate([item[-12:] for item in img_for_band_names[:1]])) # get bandnames of all bands to use in function below

# img.getDownloadURL({
#     'bands': band_names,
#     'region': AOI,
#     'filePerBand': False
# })

# i still want just one image, not the median. My idea is to sum up all the pixels in the mask band for each image
# in the collection, then sort by ascending on the sums, so at the top of the list is the image with the 
# least number of masked pixels. Having one image lets me determine prior rainfall to the date of image aquisition to 
# better replicate the results of Dodin et al. 2021

# Compute and add multiple bands, 1,2,4,5, note band 3 needs a different method since it is not just a simple normalized difference between two bands.
# idx1 = img.normalizedDifference(['B11', 'B8A']).rename('EOMI_1')
# idx2 = img.normalizedDifference(['B12', 'B4']).rename('EOMI_2')
# bandNameExp = '((b("B11") - b("B8A")) + (b("B12") - b("B4"))) / (b("B11") + b("B8A") + b("B12") + b("B4"))'
# idx3 = img.expression(bandNameExp).rename('EOMI_3')
# idx4 = img.normalizedDifference(['B11', 'B4']).rename('EOMI_4')
# idx5 = img.normalizedDifference(['B11', 'B12']).rename('EOMI_5')
# newBands = ee.Image([idx1, idx2, idx3, idx4, idx5])

# # add bands to image
# img_EOMI = img.addBands(newBands).getInfo()



# ee_array_to_df(a19_pre_list,['B12', 'B11', 'B8A', 'B4'])
# export image as csv to drive
# ee.batch.Export.image.toDrive(img_EOMI).start()


## Find out dates of best images using geemap
# # find out dates of best images
# print(geemap.image_props(s2_col_pre).get('IMAGE_DATE').getInfo())
# print(geemap.image_props(s2_col_post).get('IMAGE_DATE').getInfo())
# # print(geemap.image_props(s2_col_pre).getInfo())